In [1]:
from transformers import (
    AutoModelForSequenceClassification,
    AutoTokenizer,
    Trainer,
    TrainingArguments,
)

from sklearn.metrics import (
    accuracy_score,
    f1_score,
    confusion_matrix,
    ConfusionMatrixDisplay,
)
from datasets import load_dataset


from sklearn.model_selection import train_test_split
import pandas as pd
import time
#import matplotlib.pyplot as plt
import sys, os
import torch

print(sys.executable)


c:\Users\Timia\anaconda3\envs\algtrading\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


c:\Users\Timia\anaconda3\envs\algtrading\python.exe


In [ ]:
# Load the dataset
test_df = pd.read_csv("test.csv")

In [ ]:

start = time.time()

model_path = "results/checkpoint-1107/"

"nlptown/bert-base-multilingual-uncased-sentiment"
# tokenizer = AutoTokenizer.from_pretrained(model_path)
tokenizer = AutoTokenizer.from_pretrained("NbAiLab/nb-bert-base")
model = AutoModelForSequenceClassification.from_pretrained(model_path)


def get_sentiment(text):
    inputs = tokenizer(
        text, return_tensors="pt", truncation=True, padding="max_length", max_length=512
    )
    outputs = model(**inputs)
    probabilities = torch.nn.functional.softmax(outputs.logits, dim=-1)
    return probabilities.argmax().item()  # Gets highest probability class index



test_df["predicted_label"] = test_df["norwegian_sentence"].apply(get_sentiment)
print(test_df.head())  # View predictions
print("It took", time.time() - start, "seconds.")

In [ ]:
start = time.time()


# Convert labels to numeric format (if necessary)
label_mapping = {"Negative": 0, "Neutral": 1, "Positive": 2}
test_df["true_label"] = test_df["label"].map(label_mapping)
print(test_df.isnull().sum())  # Shows count of missing values per column
print(test_df[test_df["true_label"].isnull()])  # Displays rows with missing labels
test_df = test_df.dropna(subset=["true_label"])

# Evaluate accuracy
accuracy = accuracy_score(test_df["true_label"], test_df["predicted_label"])
print(f"Model Accuracy: {accuracy:.2f}")

# Print detailed performance report
print(
    classification_report(
        test_df["true_label"],
        test_df["predicted_label"],
        target_names=["Negative", "Neutral", "Positive"],
    )
)

print("It took", time.time() - start, "seconds.")



In [ ]:
output_dir = f"{time.strftime("%Y%m%d-%H%M%S")}"
model_path = "sentiment_model-20250609-231802"


model = AutoModelForSequenceClassification.from_pretrained(model_path)
tokenizer = AutoTokenizer.from_pretrained("NbAiLab/nb-bert-base")


# Load dataset
dataset = load_dataset(
    "csv",
    data_files={
        "train": "train.csv",
        "validation": "validation.csv",
        "test": "test.csv",
    },
)




# Tokenize data
def tokenize_function(examples):
    return tokenizer(
        examples["norwegian_sentence"],
        padding="max_length",
        truncation=True,
        max_length=256,
    )


tokenized_datasets = dataset.map(tokenize_function, batched=True)



def predict(batch):
    inputs = {k: torch.tensor(batch[k]) for k in batch if k in tokenizer.model_input_names}
    with torch.no_grad():
        outputs = model(**inputs)
    predictions = outputs.logits.argmax(dim=-1).tolist()
    return {"predicted_label": predictions}

predicted_dataset = tokenized_datasets.map(predict, batched=True)

df_predictions = predicted_dataset["test"].to_pandas()
print(df_predictions.head())


Map: 100%|██████████| 633/633 [02:06<00:00,  5.00 examples/s]

                                    english_sentence  \
0  The EUR17m contract includes both design and c...   
1  Konecranes Oyj KCR1V FH fell 5.5 percent to 20...   
2  Earnings per share for the quarter were also h...   
3  The Estonian beverages maker A. Le Coq today b...   
4  It is being developed by Symbian , the softwar...   

                                  norwegian_sentence  label  \
0  Kontrakten EUR17m omfatter både design- og byg...      1   
1  Konecranes Oyj KCR1V FH falt 5.5 prosent til 2...      0   
2  Inntekter per aksje for kvartalet var også høy...      2   
3  Den estiske drikkevareprodusenten A. Le Coq br...      1   
4  Det blir utviklet av Symbian , programvare lis...      1   

                                           input_ids  \
0  [101, 30186, 82942, 26146, 17751, 11273, 34264...   
1  [101, 30186, 54881, 27275, 10107, 35978, 10418...   
2  [101, 36702, 15186, 10877, 10178, 19647, 31205...   
3  [101, 10235, 23933, 11275, 17094, 64907, 18085...   
4  [

' print("\n\nTest set Metrics:")\nfor key, value in test_results.metrics.items():\n    print(f"{key}: {value}")\n\n\ny_true = test_results.label_ids\ny_pred = test_results.predictions.argmax(axis=-1)\n\ncm = confusion_matrix(y_true, y_pred)\ndisp = ConfusionMatrixDisplay(confusion_matrix=cm)\ndisp.plot()\nplt.title("Confusion Matrix")\nplt.savefig(f"confusion_matrix-{output_dir}.png")\nplt.close() '

In [14]:
dataset["test"]

Dataset({
    features: ['english_sentence', 'norwegian_sentence', 'label'],
    num_rows: 633
})

In [23]:
mistakes = 0
labels = ["Negative", "Neutral", "Positive"]

for index, row in dataset["test"].to_pandas().iterrows():
    text = row["norwegian_sentence"]  # Ensure column name is correct
    label = row["label"]
    
    inputs = tokenizer(text, return_tensors="pt")

    with torch.no_grad():
        outputs = model(**inputs)
    
    predicted_class = outputs.logits.argmax().item()
    if label != predicted_class:
        mistakes += 1
        print(f"Input {index}: {text}\nLabel (expected vs predicted): {labels[label]} | {labels[predicted_class]}\n\n")    
        time.sleep(1)  # Pause for 1 second before showing the next line
    #input("Press Enter to see the next line...")

print(f"Total mistakes: {mistakes} out of {len(dataset['test'])} ({(mistakes / len(dataset['test'])) * 100:.2f}%)")

Input 14: Kaupthing spår at det finsk-svenske Stora Enso vil stenge sin fabrikk i Reisholz i Tyskland.
Label (expected vs predicted): Neutral | Negative


Input 15: Elcoteq har en dokumentert merittliste innen elektronikkproduksjonstjenester (EMS) og et globalt fabrikknettverk kombinert med moderne produksjonsutstyr og konsistente systemer og prosesser.
Label (expected vs predicted): Positive | Neutral


Input 27: vil tilby en ytterligere rabatt på mellom 25 % og 50 % på utvalgte bøker.
Label (expected vs predicted): Neutral | Positive


Input 28: Han forventer imidlertid at bankene tilbyr alternativ finansiering.
Label (expected vs predicted): Positive | Neutral


Input 40: Aksjer i Royal og Sun Alliance fortsatte å skli tilbake fra en 12-måneders topp på 172p forrige måned, etter at en potensiell kjøper utelukket seg selv fra et overtakelsestilbud.
Label (expected vs predicted): Negative | Positive


Input 45: Partene har også blitt enige om alternativer som tillater UH å i stor grad

In [29]:
sentences = [
"DNB rapporterer stabile forhold.",
"Driftsresultat før avskrivninger (ebitda) var på stedet hvil: 5,4 millioner euro.",
"Også Lemmink+ñinen 's fortjeneste for regnskapsperioden gikk opp til EUR 3.1 mn fra EUR -24.5 mn et år siden .",
"For tremånedersperioden desember 2024 til februar 2025 var det en oppgang på 1,3 prosent sammenlignet med foregående tremånedersperiode.",
"Nel Hydrogen US, et datterselskap av Nel asa, har vunnet en kontrakt med Collins Aerospace for levering av oksygen til US Navy til bruk i den amerikanske marinens ubåter.",
"Equinor, Shell og TotalEnergies investerer til sammen 7,5 nye milliarder i CO2-lageret Northern Lights på Kollsnes i Øygarden kommune.",
"I en børsmelding torsdag morgen, skriver Hydrogenpro at det har mottatt en ordre på sine høytrykks alkaline elektrolyser til en verdi av 2,5 millioner dollar, eller rett i overkant av 26 millioner kroner.",
"Oslo Børs-noterte DNO melder onsdag morgen om et funn i letebrønnen Kjøttkake, der olje- og gasselskapet eier 40 prosent.",
"Frankfurt-børsen har gått som en kule i år og nøkkelindeksen Dax er opp nesten 16 prosent siden nyttår.",
"Den kinesiske elbilprodusenten BYD fikk 777 milliarder yuan – 1126 milliarder kroner – i inntekter i 2024, mer enn den amerikanske erkerivalen Tesla.",
]